In [1]:
#making necessary imports
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.base import TransformerMixin
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,precision_score, recall_score,f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
pd.set_option('display.max_columns', 500)

In [62]:
df = pd.read_excel('Employee Data Train.xlsx')

In [64]:
X = df.drop(['Termination_Status'], axis=1)
y = df['Termination_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=df['Termination_Status'], random_state=1)

In [65]:
num_features = list(X.select_dtypes(include=['int64']).columns)
cat_features = list(X.select_dtypes(include=['object']).columns)

In [66]:
pipeline = Pipeline([("robust_scaler", RobustScaler())])

#using column transformer to deal with categorical columns using OneHotEncoder
preprocessor = ColumnTransformer([
        ("num", pipeline, num_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features)
    ])

In [9]:
lr = Pipeline([('preprocessor', preprocessor),
                                ('classifier', LogisticRegression())])
param_grid = [
    {'classifier__solver': ['liblinear'], 'classifier__penalty': ['l1', 'l2']},
    {'classifier__solver': ['lbfgs', 'sag', 'saga'], 'classifier__penalty': ['none', 'l2']}
]

lr_grid_search = GridSearchCV(lr, param_grid, cv=10, scoring="accuracy")
lr_grid_search.fit(X_train, y_train)
y_pred = lr_grid_search.predict(X_test)
print("With best parameters: ", lr_grid_search.best_params_, ", accuracy = ", accuracy_score(y_pred, y_test))

/Users/himanshujat/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/himanshujat/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

/Users/himanshujat/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/himanshujat/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/himanshujat/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/himanshujat/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/himanshujat/miniforge3/lib/python3.9/site-packages/sklearn/linear

/Users/himanshujat/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/himanshujat/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/himanshujat/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/himanshujat/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/himanshujat/miniforge3/lib/python3.9/site-packages/sklearn/linear

With best parameters:  {'classifier__penalty': 'l2', 'classifier__solver': 'liblinear'} , accuracy =  0.8129786376461104


In [10]:
class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

gnb = Pipeline([('preprocessor', preprocessor), ('to_dense', DenseTransformer()), 
                                ('classifier', GaussianNB())])
param_grid = [
    {'classifier__var_smoothing': [1e-11, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2]}
]

gnb_grid_search = GridSearchCV(gnb, param_grid, cv=10, scoring="accuracy")
gnb_grid_search.fit(X_train, y_train)
y_pred = gnb_grid_search.predict(X_test)
print("With best parameters: ", gnb_grid_search.best_params_, ", accuracy = ", accuracy_score(y_pred, y_test))

With best parameters:  {'classifier__var_smoothing': 1e-06} , accuracy =  0.7698508665860541


In [11]:
knn = Pipeline([('preprocessor', preprocessor),
                                ('classifier', KNeighborsClassifier())])
param_grid = [
    {'classifier__n_neighbors': [4,5,6,7,8,9,10]}
]

knn_grid_search = GridSearchCV(knn, param_grid, cv=10, scoring="accuracy")
knn_grid_search.fit(X_train, y_train)
y_pred = knn_grid_search.predict(X_test)
print("With best parameters: ", knn_grid_search.best_params_, ", accuracy = ", accuracy_score(y_pred, y_test))

With best parameters:  {'classifier__n_neighbors': 9} , accuracy =  0.7972591696896413


In [ ]:
rf = Pipeline([('preprocessor', preprocessor),
                                ('classifier', RandomForestClassifier())])
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

param_grid = [
    {'classifier__n_estimators': n_estimators},
    {'classifier__max_features': max_features},
    {'classifier__max_depth': max_depth},
    {'classifier__min_samples_split': min_samples_split},
    {'classifier__min_samples_leaf': min_samples_leaf},
    {'classifier__bootstrap': bootstrap}
]

rf_grid_search = GridSearchCV(rf, param_grid, cv=5, scoring="accuracy")
rf_grid_search.fit(X_train, y_train)
y_pred = rf_grid_search.predict(X_test)
print("With best parameters: ", rf_grid_search.best_params_, ", accuracy = ", accuracy_score(y_pred, y_test))

In [67]:
svc = Pipeline([('preprocessor', preprocessor),
                                ('classifier', SVC())])
param_grid = [{'classifier__C': [10]}
    #{'classifier__C': [0.5, 0.75, 1, 2, 3]}
]

svc_grid_search = GridSearchCV(svc, param_grid, cv=10, scoring="accuracy")
svc_grid_search.fit(X_train, y_train)
y_pred = svc_grid_search.predict(X_test)
print("With best parameters: ", svc_grid_search.best_params_, ", accuracy = ", accuracy_score(y_pred, y_test))

With best parameters:  {'classifier__C': 10} , accuracy =  0.8192211504108611


In [68]:
precision, recall, f1, support = precision_recall_fscore_support(y_test, y_pred, average='macro')
print('Precision:', precision)
print('Recall:', recall)
print('f1:', f1)

Precision: 0.8203484549638396
Recall: 0.8182137568105932
f1: 0.8186059843500731


In [69]:
test = pd.read_excel('Employee Data Test.xlsx')
test

,Termination_Status,employee_id,location_tier,location_cost,location_reporting,restated_LOB,tenure,age,qc_non_qc,is_people_manager,job_family_group,gender,management_level_category,management_chain_level_employee,Job_level,is_DQC,Stock_Eligible,remote_location,Plan_Category,compensation_usd,competitive_base_pay_market_position,competitive_total_target_cash_market_position,base_pay_compa_ratio,total_target_cash_compa_ratio,Time_in_Job_Profile,bonus_eligible,promotion_count_rolling_2_years,transfer_count_rolling_2_years,change_job_type_count_rolling_2_years,time_since_last_promotion_in_years,time_since_last_transfer_in_years,time_since_last_change_job_type_in_years,Change_in_Individual_management_level,Change_in_job_level,current_vs_average_compensation_job_level,current_vs_average_tenure_job_level,current_vs_average_tenure,%_Abover_or_Below_Average_Compensation_job_level,%_Abover_or_Below_Average_Tenure_job_level,%_Abover_or_Below_Average_Tenure_Company,termination_date
0,No,5132,Tier 1,Low-Cost,Bangalore,Engineering,15.42,37,NON QC,No,Engineering,Male,Individual Contributor/Support,5,IC5,Unknown,Yes,No,VCIP Plan,91827.41,3,3,1.093,1.097,3.832991,Yes,0,0,0,3.835616,15.42,15.42,No,No,Higher than average,Higher than average,Higher than average,21.71,129.67,257.84,NaT
1,No,40993,Tier 1,High-Cost,Raleigh,Engineering,0.50,33,NON QC,No,Engineering,Female,Individual Contributor/Support,5,IC4,Unknown,No,No,VCIP Plan,187000.00,2,2,0.899,0.862,0.514715,Yes,0,0,1,0.500000,0.50,0.30,No,No,Higher than average,Lower than average,Lower than average,30.92,-79.56,-88.40,NaT
2,No,37580,All Others,High-Cost,All Other EMEA,Sales & Services,2.17,50,QC,No,Sales,Male,Individual Contributor/Support,4,IC3,No,Yes,No,Comission Plan,126730.25,3,3,0.976,0.976,2.239561,Yes,0,0,0,2.170000,2.17,2.17,No,No,Higher than average,Lower than average,Lower than average,2.34,-52.28,-49.64,NaT
3,No,41054,Tier 1,Low-Cost,Bangalore,Sales & Services,0.58,27,NON QC,No,Business Operations,Male,Individual Contributor/Support,4,IC3,No,No,No,No Bonus Plan,17927.25,3,3,0.901,0.901,0.665297,No,0,0,0,0.580000,0.58,0.58,No,No,Lower than average,Lower than average,Lower than average,-9.02,-79.00,-86.54,NaT
4,No,2321,Tier 1,High-Cost,Fort Lauderdale,Engineering,19.17,44,NON QC,No,Engineering,Male,Individual Contributor/Support,5,IC6,Unknown,Yes,No,VCIP Plan,225461.45,3,3,0.964,1.002,2.335386,Yes,0,0,0,8.673972,19.17,5.07,No,No,Lower than average,Higher than average,Higher than average,-8.89,34.46,344.86,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9333,No,17632,Tier 1,High-Cost,Fort Lauderdale,Operations,7.75,35,QC,Yes,Sales,Male,People Manager,6,M3,No,Yes,No,Comission Plan,143102.14,5,3,1.431,1.020,1.336071,Yes,1,0,0,1.336986,7.75,2.21,No,Yes,Lower than average,Lower than average,Higher than average,-3.82,-14.36,79.85,NaT
9334,No,37007,All Others,High-Cost,Remote AMS,Sales & Services,2.58,41,QC,No,Sales,Male,Individual Contributor/Support,5,IC4,Yes,Yes,Yes,Comission Plan,298494.00,4,4,1.133,1.184,0.416153,Yes,0,2,0,2.580000,0.42,2.64,No,No,Higher than average,Lower than average,Lower than average,35.07,-31.49,-40.13,NaT
9335,No,35022,All Others,High-Cost,Remote AMS,Sales & Services,3.58,30,NON QC,No,Sales,Male,Individual Contributor/Support,5,IC2,No,Yes,Yes,No Bonus Plan,96000.00,1,1,0.789,0.751,0.334017,No,0,1,0,2.756164,0.33,3.59,No,No,Lower than average,Higher than average,Lower than average,-24.89,14.37,-16.92,NaT
9336,No,39984,Tier 1,High-Cost,Fort Lauderdale,Sales & Services,0.92,24,NON QC,No,Customer Success,Female,Individual Contributor/Support,3,IC1,No,No,No,No Bonus Plan,70000.00,3,3,1.072,1.072,0.993839,No,0,0,0,0.920000,0.92,0.92,No,No,Lower than average,Lower than average,Lower than average,-18.29,-48.26,-78.65,NaT


In [70]:
test.Termination_Status.value_counts()

No     9057
Yes     281
Name: Termination_Status, dtype: int64

In [71]:
test.drop(['Termination_Status', 'termination_date'], inplace=True, axis=1)
results = svc_grid_search.predict(test)

In [72]:
len(results)

9338

In [73]:
test = pd.read_excel('Employee Data Test.xlsx')
results_df = test[['Termination_Status']]
results_df['predicted'] = results

<ipython-input-73-d51be55f0c32>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df['predicted'] = results


In [74]:
results_df['match'] = results_df['Termination_Status'] == results_df['predicted']

<ipython-input-74-dfbb26a9d209>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df['match'] = results_df['Termination_Status'] == results_df['predicted']


In [75]:
results_df.loc[results_df.Termination_Status=='Yes'].shape, results_df.loc[results_df.Termination_Status=='No'].shape

((281, 3), (9057, 3))

In [76]:
results_df.loc[results_df.predicted=='Yes'].shape, results_df.loc[results_df.predicted=='No'].shape

((1363, 3), (7975, 3))

In [77]:
results_df.drop(['Termination_Status'], axis=1, inplace=True)

/Users/himanshujat/miniforge3/lib/python3.9/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [78]:
#pd.concat([results_df, test], axis=1).to_csv('results.csv')

In [79]:
final_df = pd.concat([results_df, test], axis=1)

In [80]:
final_df.match.value_counts()

True     7830
False    1508
Name: match, dtype: int64

In [81]:
final_df.shape

(9338, 43)

In [82]:
8724/9338*100

93.42471621332191

In [83]:
final_df.loc[final_df.Termination_Status == 'Yes', ['predicted', 'Termination_Status']]

,predicted,Termination_Status
63,No,Yes
73,Yes,Yes
85,Yes,Yes
114,No,Yes
152,No,Yes
...,...,...
9192,No,Yes
9198,No,Yes
9210,No,Yes
9238,No,Yes


In [84]:
final_df.loc[final_df.Termination_Status == 'Yes', ['predicted', 'Termination_Status']].predicted.value_counts()

No     213
Yes     68
Name: predicted, dtype: int64